<h1 style="font-size: 36px; color: #FFD700">6 - AUTOGRAD</h1>

Thuật toán được sử dụng nhiều khi huấn luyện mạng neural là lan truyền ngược. Trong thuật toán này, các tham số - weight đc điều chỉnh theo độ dốc của hàm loss với tham số đã cho

torch.autograd để tính toán gradient.

In [21]:
import torch

x = torch.ones(5) # một tensor đầu vào với 5 phần tử
print(f"input: {x}") 
y = torch.zeros(3) # output kì vọng
print(f"output: {y}")
w = torch.randn(5, 3, requires_grad= True) # ma trận trọng số
print(f"weight: {w}")
b = torch.randn(3, requires_grad= True) # bias
print(f"bias: {b}")
z = torch.matmul(x, w) + b # phương trình tìm z với x đầu vào và trọng số
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(f"output_pred: {z}")
print(f"loss: {loss}")

input: tensor([1., 1., 1., 1., 1.])
output: tensor([0., 0., 0.])
weight: tensor([[-0.0051, -0.3425, -0.6627],
        [ 0.3125, -0.3847,  1.8364],
        [-0.3172,  0.0888, -1.0711],
        [ 1.3015,  1.0996, -0.8059],
        [ 0.2707,  0.2941, -0.1797]], requires_grad=True)
bias: tensor([ 1.4978, -0.0189, -0.6739], requires_grad=True)
output_pred: tensor([ 3.0602,  0.7364, -1.5569], grad_fn=<AddBackward0>)
loss: 1.4749889373779297


Ta có thể thiết laappj requires_grad khi tạo tensor hoặc sử dụng method x.requires_grad_(True)

Đồ thị tính toán:

<img src="https://pytorch.org/tutorials/_images/comp-graph.png" alt="Ảnh có nền trắng" style="background-color: white;">


w, b là các parameter cần tối ứu => phải đặt requires_grad = True

- Thuộc tính - grad_fn, ta áp dụng cho tensor là một đối tượng của lớp Function - trỏ đến một đối tượng của một lớp con kế thừa từ torch.autograd.Function. 

- đối tượng này là hàm(matmul, +) đã tạo ra tensor khi lan truyền tiến - forward

- Nó có chứa logic để thực hiện backward

=> grad_fn lưu trữ đối tượng mô ta các phép toán hình thành nên tensor đó

In [18]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x70342df602e0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x70342795e980>


Gradient

Để tối ưu, ta tính gradient của loss function của chúng ta với các parameter. Cụ thể ta cần
$\frac{\partial \text{loss}}{\partial w}$
$\frac{\partial \text{loss}}{\partial b}$
của một số giá trị cố định của x và y

=> để tính ta gọi loss.backward()



In [22]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3184, 0.2254, 0.0580],
        [0.3184, 0.2254, 0.0580],
        [0.3184, 0.2254, 0.0580],
        [0.3184, 0.2254, 0.0580],
        [0.3184, 0.2254, 0.0580]])
tensor([0.3184, 0.2254, 0.0580])


leaf_node + requires_grad=True => tự động có .grad sau khi gọi loss.backward. Còn lại gradient không khả dụng 
- leaf_node: Là những tensor do ta tạo ra, không phải kq của phép toán khác. Vd: x, y, w, b

ta chỉ có thể thực hiện grad bằng cách dùng backward một lần trên một đồ thị nhất định vì hiệu suất. 
- retain_graph=True => thực hiện nhiều backward trên cùng một đồ thị 

Vô hiệu hóa theo dõi gradient

Khi tạo tensor vs requires_grad=True:
- Theo dõi tất cả các phép toán ta thực hiện trên tensor đó
- Dùng để xây dựng đồ thị tính toán => phục vụ Backward

Khi không cần gradient
- Mô hình train xong
- Chỉ muốn dự đoán đầu ra từ dữ liệu mới

=> theo dõi phép toán và tính gradient là vô ích => tốn resource

===> Bao quanh mã tính toán bằng block torch.no_grad() / detach()

In [23]:
# bọc quanh mã tính toán bằng block torch.no_grad()
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


In [24]:
# detach()
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


Lý do khác nên tắt tính năng theo dõi grad:
- Để đánh dấu một số tham số trong mạng neural là tham số cố định => khi chỉ muốn fine-tuning pretrained-model, đóng băng một số lớp
- Các phép tính trên tensor khi không theo dõi grad sẽ hiệu quả hơn => tăng tốc độ tính toán

Computational Graphs

Autograd là hệ thống tự động tính đạo hàm, khi đó:
- Thực hiện phép tính như bình thường → ví dụ: nhân ma trận, cộng, sigmoid, loss, v.v
- Ghi lại các phép toán đó vào một đồ thị có hướng (DAG)
    - Mỗi phép toán là một nút Function trong đồ thị
    - Mỗi tensor trung gian nhớ lại phép toán đã tạo ra nó thông qua .grad_fn


Cấu trúc đồ thị:
- Leaf: là các tensor đầu vào - ta tạo ra: x, w, b
- Root: là tensor đầu ra cuối cùng, thường là loss

Đồ thị DAG => không có vòng lặp

Khi gọi .backward() tại root(loss.backward()), khi đó:
1. Bắt đầu từ loss - gốc của DAG
2. Tính grad của từng phép toán bằng .grad_fn
3. Tích lũy grad vào thuộc tính .grad của các tensor có requires_grad=True
4. Lan truyền ngược xuống tới các tensor lá bằng quy chain rule

Ghi chú: 

DAG là dynamic - sau mỗi lần gọi .backward, autograd tự xây lại một đồ thị mới. Sau khi backward xong thì đồ thị cũ bị xóa. Đây là điều cho phép ta sử dụng câu lệnh điều khiển luồng (if, for, while) trong môt hình. Ta có thể thay đổi shape, size, operator ở mỗi lần lặp nếu cần mà autograd vẫn đoạt động chính xác.

In [27]:
import torch

x = torch.tensor([2.0], requires_grad=True)

for i in range(3):
    if i % 2 == 0:
        y = x * 2      # Lần 1 & 3: phép nhân
    else:
        y = x ** 2     # Lần 2: phép lũy thừa

    # nếu DAG tĩnh => lỗi hoặc kq không thay đổi
    print(f"--- Lần {i+1} ---")
    print(f"Hàm sử dụng: {'x * 2' if i % 2 == 0 else 'x ** 2'}")
    
    y.backward()
    print(f"x.grad = {x.grad}\n")
    
    x.grad.zero_()  # Reset gradient để lặp tiếp


--- Lần 1 ---
Hàm sử dụng: x * 2
x.grad = tensor([2.])

--- Lần 2 ---
Hàm sử dụng: x ** 2
x.grad = tensor([4.])

--- Lần 3 ---
Hàm sử dụng: x * 2
x.grad = tensor([2.])



Ta thấy lần 2 thì kết quả grad vẫn đúng với bản chất grad chứ không giống như kq của lần 1 => đồ thị sẽ cập nhật sau sau mỗi lần gọi backward

Tensor gradients and Jacobian Products

Trong trường hợp ta có một hàm loss vô hướng và ta cần tính toán gradient theo một số tham số, có những trường hợp hàm đầu ra là một tensor tùy ý thì Pytorch cho ta tính toán tích Jacobian chứ không phải grad.

$ \text{Với một hàm vector } \vec{y} = f(\vec{x}), \text{ trong đó } \vec{x} = \langle x_1, \dots, x_n \rangle, \vec{y} = \langle y_1, \dots, y_m \rangle, \text{ gradient của } \vec{y} \text{ theo } \vec{x} \text{ được cho bởi ma trận Jacobian:} $


$$
J = \begin{pmatrix}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial y_m}{\partial x_1} & \cdots & \frac{\partial y_m}{\partial x_n}
\end{pmatrix}
$$


thay vì tính toán ma trận Jacobian, Pytorch cho phép tính toán Jacobian product $\vec{v}^T \cdot J$ cho một vector đầu vào nhất định $\vec{v} = \langle v_1, \dots, v_n \rangle$. Điều này đạt được bằng cách gọi .backward(v). shape v phải giống shape tensor ban đầu.


In [29]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp + 1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")

out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


Khi ta gọi backward lần thứ hai với cùng một đối số, giá trị của grad sẽ khác vì khi backward lan truyền, Pytorch tích lũy các grad - giá trị các grad được thêm vào grad thuộc  tính của tất cả các nút lá của đồ thị thuật toán. Nếu muốn tính toán grad thích hợp thì cần phải xóa thuộc tính grad trước. Thực tế thì optimizer sẽ giúp ta thực hiện

Ghi chú: Trước đây, backward không có tham số. Điều này tương đương backward(torch.tensor(1.0)) đây hữ ích khi tính grad trong trường hợp hàm có giá trị vô hướng. Vd như loss trong qtrinh train neural network